In [ ]:
import sys
sys.path.append('../../')

In [2]:
import pandas as pd
import pickle
import numpy as np
from holsteinlib import functions

# MiniRocket
from sktime.transformations.panel.rocket import MiniRocketMultivariateVariable

In [3]:
def duplicate_check(arr_1, arr_2):
    set_arr_2 = set(arr_2)
    
    for x in arr_1:
        if x in set_arr_2:
            return True
    
    return False

In [4]:
def get_data(calf_id):
    
    X,y = functions.combine_class_data(window_dataset[calf_id])
        
    return X, y

In [5]:
dataset_path = '../../Datasets/SixBehaviourClassification/six_label_window_dataset_v1.pickle'
calf_set_info_path = '../../Datasets/SixBehaviourClassification/six_label_calf_split_info.pkl'

feature_data_save_parent_path = '../../Datasets/SixBehaviourClassification/ROCKET/'
per_calf_data_index_in_X_train_save_path = '../../Datasets/SixBehaviourClassification/ROCKET/'

In [6]:
# dataset
with open(dataset_path, 'rb') as f:
    window_dataset = pickle.load(f)
    
# calf train:valid:test set information
with open(calf_set_info_path, 'rb') as f:
    calf_set_info = pickle.load(f)

In [7]:
calf_set_info.keys()

dict_keys(['all_calves', 'test_calves', 'valid_calf_id_sets'])

In [8]:
all_calf_ids = calf_set_info['all_calves']
test_calf_ids = calf_set_info['test_calves']

total_train_calf_ids = [calf for calf in all_calf_ids if not calf in test_calf_ids]

# Feature Generation

## Combine Train Data

In [9]:
X_train, y_train = [], []

calf_X_train_indexes = {}

index_tracker = 0

for calf_id in all_calf_ids:
    if calf_id not in test_calf_ids:
        sub_X, sub_y = get_data(calf_id)
        
        X_train.extend(sub_X)
        y_train.extend(sub_y)
        
        # track the position of data per calf in X_train
        calf_X_train_indexes[calf_id] = [i for i in range (index_tracker, index_tracker+len(sub_X))]
        
        index_tracker = index_tracker + len(sub_X)

In [10]:
# save per calf indexes in X_train
with open(per_calf_data_index_in_X_train_save_path + 'calf_data_indexes_in_X_train.pickle', 'wb') as handle:
    pickle.dump(calf_X_train_indexes, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Combine Test Data

In [11]:
X_test, y_test = [], []

for calf_id in test_calf_ids:
    sub_X, sub_y = get_data(calf_id)

    X_test.extend(sub_X)
    y_test.extend(sub_y)

## Generating Features

In [12]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

In [13]:
%%time

trf = MiniRocketMultivariateVariable(num_kernels=10000) 
trf.fit(X_train_df) 
X_train_features = trf.transform(X_train_df) 
X_test_features = trf.transform(X_test_df) 

CPU times: user 2min 28s, sys: 2.23 s, total: 2min 30s
Wall time: 2min 30s


In [14]:
np.save(feature_data_save_parent_path + 'X_train_features', X_train_features)
np.save(feature_data_save_parent_path + 'X_test_features', X_test_features)

np.save(feature_data_save_parent_path + 'y_train', y_train)
np.save(feature_data_save_parent_path + 'y_test', y_test)